In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load and parse the XML file
tree = ET.parse('queueinfo.xml')
root = tree.getroot()

In [ ]:
# Extract data from XML
data = []
for timestep in root.findall('data'):
    step = float(timestep.get('timestep'))
    for lane in timestep.find('lanes').findall('lane'):
        lane_id = lane.get('id')
        queueing_length = float(lane.get('queueing_length'))
        queueing_time = float(lane.get('queueing_time'))
        data.append((step, lane_id, queueing_length, queueing_time))

data

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data, columns=['step', 'lane_id', 'queueing_length', 'queueing_time'])
df

In [ ]:
# 전처리 : lane_id 가 c로 시작하는 행 제거
df = df[~df['lane_id'].str.startswith('c')]
df

In [ ]:
# lane_id 가 1c_0, 1c_1, 1c_2, 1c_3, 1c_4 인 행을 1c 로 통일. 이때, 같은 time의 queueing_length 는 해당 time의 각 lane_id 의 queueing_length 의 평균으로 대체
df['lane_id'] = df['lane_id'].apply(lambda x: x.split('_')[0])
df = df.groupby(['step', 'lane_id']).mean().reset_index()
df

In [ ]:
# Plot the data
lanes = df['lane_id'].unique()
fig, ax = plt.subplots()

for lane in lanes:
    lane_data = df[df['lane_id'] == lane]
    ax.plot(lane_data['step'], lane_data['queueing_length'], label=lane)

ax.set_xlabel('Step (s)')
ax.set_ylabel('Queueing Length (m)')
ax.legend(title='Lane ID')
plt.title('Queueing Length Over Step by Lane')
plt.show()

In [ ]:
# lane_id별로 plot 4개의 subplot 으로 나누어 그리기 + 5차 회귀선 추가
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
for i, lane in enumerate(lanes):
    lane_data = df[df['lane_id'] == lane]
    ax = axs[i//2, i%2]
    ax.plot(lane_data['step'], lane_data['queueing_length'], label=lane)
    z = np.polyfit(lane_data['step'], lane_data['queueing_length'], 5)
    p = np.poly1d(z)
    ax.plot(lane_data['step'], p(lane_data['step']), 'r--', label='5th degree polynomial')
    ax.set_xlabel('Step (s)')
    ax.set_ylabel('Queueing Length (m)')
    ax.legend(title='Lane ID')
    ax.set_title(f'Queueing Length Over Step by Lane {lane}')
plt.show()